<a href="https://colab.research.google.com/github/jessica-guan/Python-DataSci-ML/blob/main/AWS%20Clarify%3A%20Analysis%20of%20UCI's%20Parkinson's%20Disease%20Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Homework 15: AI Ethics**
---

### **Description**:
In this notebook, we will use AWS Clarify to analyze UCI's Parkinson's Dataset.

<br>

### **Structure**
**Part 1**: [UCI's Parkinson's Dataset](#p1)



</br>




### **Cheat Sheets**
[AWS Clarify](https://docs.google.com/document/d/1eGmQBEzCt4YBgPDxWB7j6UPb1HPucIISz3k1FLGunr4/edit?usp=sharing)

<br>

**Before starting, run the code below to import all necessary functions and libraries.**


In [ ]:
#!pip install scikit-learn
!pip install --quiet smclarify

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from smclarify.bias.report import *

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.9/170.9 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.8/75.8 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

<a name="p1"></a>

---
## **Part 1: UCI's Parkinson's Dataset**
---

In this section, we are going to use Clarify to identify bias. This method is *much faster* than creating bar graphs for every column.

Here is an AWS Clarify [cheatsheet](https://docs.google.com/document/d/1PY06KZU97J-HU9nRArMH6biuoaCjPA_zm-EhIlTId2I/edit?usp=sharing) for interpretting results.



### **Problem #1.1: AWS Clarify & UCI's Parkinson's Dataset**

This dataset is composed of a range of biomedical voice measurements from 42 people with early-stage Parkinson's disease recruited to a six-month trial of a telemonitoring device for remote symptom progression monitoring. The recordings were automatically captured in the patient's homes.

Columns in the table contain subject number, subject age, subject gender, time interval from baseline recruitment date, motor UPDRS, total UPDRS, and 16 biomedical voice measures. Each row corresponds to one of 5,875 voice recording from these individuals. The main aim of the data is to predict the motor and total UPDRS scores ('motor_UPDRS' and 'total_UPDRS') from the 16 voice measures.


#### **Steps #1-2: Load the data and import packages.**


In [ ]:
url = "https://raw.githubusercontent.com/the-codingschool/TRAIN-datasets/main/parkinsons/parkinsons.csv"
df = pd.read_csv(url)
df.head()

,subject#,age,sex,test_time,motor_UPDRS,total_UPDRS,Jitter(%),Jitter(Abs),Jitter:RAP,Jitter:PPQ5,...,Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,Shimmer:APQ11,Shimmer:DDA,NHR,HNR,RPDE,DFA,PPE
0,1,72,0,5.6431,28.199,34.398,0.00662,0.000034,0.00401,0.00317,...,0.230,0.01438,0.01309,0.01662,0.04314,0.014290,21.640,0.41888,0.54842,0.16006
1,1,72,0,12.6660,28.447,34.894,0.00300,0.000017,0.00132,0.00150,...,0.179,0.00994,0.01072,0.01689,0.02982,0.011112,27.183,0.43493,0.56477,0.10810
2,1,72,0,19.6810,28.695,35.389,0.00481,0.000025,0.00205,0.00208,...,0.181,0.00734,0.00844,0.01458,0.02202,0.020220,23.047,0.46222,0.54405,0.21014
3,1,72,0,25.6470,28.905,35.810,0.00528,0.000027,0.00191,0.00264,...,0.327,0.01106,0.01265,0.01963,0.03317,0.027837,24.445,0.48730,0.57794,0.33277
4,1,72,0,33.6420,29.187,36.375,0.00335,0.000020,0.00093,0.00130,...,0.176,0.00679,0.00929,0.01819,0.02036,0.011625,26.126,0.47188,0.56122,0.19361


#### **Step #3: Denote the facet column, the label column, and the group variable.**

Set the:
* Facet column as `age`.
* Label column with `sex` as the target column and `'0'` as the positive label.
* `age` as the group variable.

In [ ]:
facet_column = FacetColumn(name='age')
label_column = LabelColumn('sex', df['sex'], ['0'])
group_variable = 'age'

#### **Step #4: Generate bias report.**


Use the cheat sheet to interpret pre-training metrics. Using this page, interpret what the following report is saying.

<br>

**Run the cell below to generate your bias report.**

In [ ]:
report = bias_report(df, facet_column, label_column, stage_type=StageType.PRE_TRAINING, group_variable=group_variable)

# use this to print your report - call it "report" for the code to work
for cl in report:
    print("\n\n","-"*35)
    print("-"*15, cl["value_or_threshold"], "-"*15)
    for metric in cl['metrics']:
        print(f"{metric['description']}: {metric['value']}")

ERROR:smclarify.bias.report:CDDL metrics failed
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/smclarify/bias/report.py", line 323, in _categorical_metric_call_wrapper
    metric_value = smclarify.bias.metrics.call_metric(
  File "/usr/local/lib/python3.10/dist-packages/smclarify/bias/metrics/__init__.py", line 27, in call_metric
    return metric(**{key: kwargs[key] for key in inspect.signature(metric).parameters.keys()})
  File "/usr/local/lib/python3.10/dist-packages/smclarify/bias/metrics/pretraining.py", line 195, in CDDL
    return common.CDD(feature, sensitive_facet_index, positive_label_index, group_variable)
  File "/usr/local/lib/python3.10/dist-packages/smclarify/bias/metrics/common.py", line 99, in CDD
    if group_variable is None or group_variable.empty:
AttributeError: 'str' object has no attribute 'empty'
ERROR:smclarify.bias.report:CDDL metrics failed
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/



 -----------------------------------
--------------- 72 ---------------
Conditional Demographic Disparity in Labels (CDDL): None
Class Imbalance (CI): 0.894468085106383
Difference in Positive Proportions in Labels (DPL): -0.3354896675651393
Jensen-Shannon Divergence (JS): 0.017021228446644887
Kullback-Liebler Divergence (KL): -0.27158859691248927
Kolmogorov-Smirnov Distance (KS): 0.3354896675651393
L-p Norm (LP): 0.3354896675651393
Total Variation Distance (TVD): 0.16774483378256966


 -----------------------------------
--------------- 58 ---------------
Conditional Demographic Disparity in Labels (CDDL): None
Class Imbalance (CI): 0.8539574468085106
Difference in Positive Proportions in Labels (DPL): -0.0008316447905136748
Jensen-Shannon Divergence (JS): 3.990149039778668e-07
Kullback-Liebler Divergence (KL): 1.5964327743804357e-06
Kolmogorov-Smirnov Distance (KS): 0.0008316447905137303
L-p Norm (LP): 0.0011761233418214096
Total Variation Distance (TVD): 0.0008316447905137025


 --

#### **Step #5: Look at the imbalance.**

Use the cheat sheet provided to interpret what the bias report is telling you.

#End of notebook
---
© 2024 The Coding School, All rights reserved